# Reading characters on license plate

### Initialization

In [20]:
# Import packages
import torch
import torch.nn as nn
import torchvision.models as models
from PIL import Image
from torchvision import transforms

In [21]:
# Pipeline to prepare the images to feed the model

# Define image size
IMAGE_HEIGHT = 263 
IMAGE_WIDTH = 800  

# Define the transformations
preprocess = transforms.Compose([
    transforms.Resize((IMAGE_HEIGHT, IMAGE_WIDTH)),  # Resize image
    transforms.ToTensor(),  # Convert image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean
                         std=[0.229, 0.224, 0.225])  # Normalize with ImageNet std
])

def process_img(path):
    # Load the image
    image = Image.open(path).convert('RGB')  # Ensure it's RGB

    # Apply transformations
    image_tensor = preprocess(image)

    # Add a batch dimension (for a single image)
    image_tensor = image_tensor.unsqueeze(0)  # Shape: [1, 3, 80, 224]

    return image_tensor

### Implement model

In [17]:
# Define the number of classes and length of the license plate
NUM_CLASSES = 36   # 26 letters + 10 digits for alphanumeric characters
NUM_CHARACTERS = 7 # Assuming license plates have 7 characters


# Load the pre-trained VGG16 model and freeze its earlier layers
vgg16 = models.vgg16(weights='VGG16_Weights.DEFAULT')
for param in vgg16.features.parameters():
    param.requires_grad = False


# Helper function to compute the flattened output size after passing through VGG16
def get_flattened_size(input_height, input_width):    
    dummy_input = torch.rand(1, 3, input_height, input_width)
    with torch.no_grad():
        features = vgg16.features(dummy_input)
    flattened_size = features.view(1, -1).size(1)
    return flattened_size

# Get the flattened size for the given image dimensions
flattened_size = get_flattened_size(IMAGE_HEIGHT, IMAGE_WIDTH)

In [18]:
# Model

class ReadPlate(nn.Module):
    def __init__(self, flattened, num_characters, num_classes):
        super(ReadPlate, self).__init__()
        # Use the pretrained VGG16 model up to the feature extractor
        self.vgg16_features = nn.Sequential(*list(vgg16.features.children()))
        
        # Custom classifier for each character in the license plate (7 outputs for a 7-character plate)
        self.classifiers = nn.ModuleList([
            nn.Sequential(
                nn.Flatten(),
                nn.Linear(flattened, 128),
                nn.ReLU(),
                nn.Linear(128, 128),
                nn.ReLU(),
                nn.Linear(128, num_classes),  # Each character output (36 classes for digits and letters)
                nn.LogSoftmax(dim=1) 
            ) for _ in range(num_characters) # Create a different classifier for each character
        ])

    def forward(self, x):
        x = self.vgg16_features(x)  # Pass through VGG16 feature extractor
        x = x.view(x.size(0), -1)   # Flatten the output
        
        # Pass them through each classifier for each character in the license plate
        outputs = [classifier(x) for classifier in self.classifiers]        
        return outputs

# Instantiate the model (7 characters, 36 possible classes)
model = ReadPlate(flattened=flattened_size, num_characters=NUM_CHARACTERS, num_classes=NUM_CLASSES)

Source: https://github.com/ramyh08/Licence-Plate-recognition-CNN/tree/main

### Inference

In [19]:
# model_output is the list of tensors produced by the model
img = 'C:/Users/User/Desktop/Assignatures/Vision and Learning/License Plate/vision-and-learning/P_G_12492604_01539.jpeg'
img_tensor = process_img(img) 
model_output = model(img_tensor)

# Define a lookup table for characters
characters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

# Initialize an empty string to hold the license plate result
license_plate = ''

# Obtain each character of the license plate
for char_output in model_output:    
    # Get the index of the highest probability class
    predicted_class = torch.argmax(char_output, dim=1) 
    
    # Map the predicted class index to the corresponding character
    predicted_character = characters[predicted_class.item()]    
    
    license_plate += predicted_character # Write the license plate

print(f"Predicted License Plate: {license_plate}")

Predicted License Plate: BSS5M3I
